In [ ]:
# CREATE TABLE Statements
"""
CREATE TABLE table_name (
  1. column_name data_type [NOT NULL] [DEFAULT default_value],
  2. [PRIMARY KEY (column_list)],
  3. [UNIQUE (column_list)],
  4. [FOREIGN KEY (column_list) REFERENCES other_table(other_column_list)]
   [ON DELETE option [ON UPDATE option]],
  5. [CONSTRAINT constraint_name CHECK (condition)]
);
"""

"""
1. []는 option을 의미함
2. ;로 마침
3. 대소문자 구분안함. 단 키워든느 대문자로 하는게 convention
4. NOT NULL는 값을 넣도록 보장해줌
5. DEFAULT는 value가 없으면 자동으로 값을 넣어줌
"""

"""
각 column은 data type, NULL 여부, default에 대한 정보를 가짐
"""

"""
CHAR: 고정된 길이
VARCHAR: 가변 길이

NUMERIC(p,s), DECIMAL(p,s): p는 총 숫자의 길이, s는 소수점 이하 갯수
REAL: 4Bytes로 부동소수점 저장함
"""

"""
PRIMARY KEY
- NULL이면 안됨
- 복합키로 여러 column이 합쳐져서 구성됨

UNIQUE
- 지정한 컬럼에 중복 값 저장 안되도록함
- NULL은 여러개 가능 및 허

FOREIGN KEY
- ON DELETE option이 있음 (행이 사라질때 작동됨)
- 외래키로 참조할 때 관련 행이 사라지면 동작함 (전체 행이 아니라 하나의 행이라도 삭제되면 동작)
1. ON DELETE NO ACTION: 삭제 방지
2. ON DELETE CASCADE: 자동적으로 삭제
3. ON DELETE SET NULL: NULL로 설정
4. ON DELETE SET DEFAULT: DEFAULT 값으로 설정

CHECK
- INSERT, UPDATE 작동 시 동작
- CONSTRAINT와 함께 사용되면 제약 조건에 이름을 주어 변수처럼 사용가능

"""

'\nPRIMARY KEY\n- NULL이면 안됨\n- 복합키로 여러 column이 합쳐져서 구성됨\n\nUNIQUE\n- 지정한 컬럼에 중복 값 저장 안되도록함\n- NULL은 여러개 가능 및 허\n\nFOREIGN KEY\n- ON DELETE option이 있음 (행이 사라질때 작동됨)\n- 외래키로 참조할 때 관련 행이 사라지면 동작함 (전체 행이 아니라 하나의 행이라도 삭제되면 동작)\n1. ON DELETE NO ACTION: 삭제 방지\n2. ON DELETE CASCADE: 자동적으로 삭제\n3. ON DELETE SET NULL: NULL로 설정\n4. ON DELETE SET DEFAULT: DEFAULT 값으로 설정\n\nCHECK\n- INSERT, UPDATE 작동 시 동작\n- CONSTRAINT와 함께 사용되면 제약 조건에 이름을 주어 변수처럼 사용가능\n\n'

In [ ]:
# STEP 1: Install MySQL Server
!apt-get update
!apt-get install -y mysql-server

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,742 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,019 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 12.2 MB in 6s (1,904 kB/s)
Reading package lists... Done
W: Skipping acquire of conf

In [ ]:
# STEP 2: Start MySQL Server
!service mysql start

 * Starting MySQL database server mysqld
   ...done.


In [ ]:
# STEP 3: Set the Root Password
# 비밀번호 없이 ROOT 접근함
# FLUSH PRIVILEGES -> 변경 사항 즉시 적용
!sudo mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY ''; FLUSH PRIVILEGES;"

In [ ]:
# STEP 4: Install the Python MySQL Connector
!pip install mysql-connector-python

In [ ]:
# STEP 5: Connect and Use MySQL in Python
import mysql.connector

# Connect to MySQL server
conn = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = ""
)

cursor = conn.cursor()

# Create a database
cursor.execute("CREATE DATABASE IF NOT EXISTS testdb")
cursor.execute("USE testdb") # testdb를 사용할거임

In [ ]:
# Example-1: Writing a CREATE TABLE Statement for a Customer Table
# CustomerID, CustomerName, Age, Grade, Job, Points
# CustomerID는 Primary Key
# CustomerName과 Grade는 null이면 안됨
# Points는 DEFAULT로 0을 가짐

cursor.execute("DROP TABLE IF EXISTS Customer")

create_table_query = """
 CREATE TABLE Customer(
  CustomerID VARCHAR(20) NOT NULL,
  CustomerName VARCHAR(20) NOT NULL,
  Age INT,
  Grade VARCHAR(10) NOT NULL,
  Job VARCHAR(20),
  Points INT DEFAULT 0,
  PRIMARY KEY (CustomerID)
 );
"""

cursor.execute(create_table_query)
print("Customer table created.")

cursor.execute("DESCRIBE Customer")
columns = cursor.fetchall()

for column in columns:
  print(column)

cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

print("List of Tables in the Database")
for table in tables:
  print(table[0])

Customer table created.
('CustomerID', 'varchar(20)', 'NO', 'PRI', None, '')
('CustomerName', 'varchar(20)', 'NO', '', None, '')
('Age', 'int', 'YES', '', None, '')
('Grade', 'varchar(10)', 'NO', '', None, '')
('Job', 'varchar(20)', 'YES', '', None, '')
('Points', 'int', 'YES', '', '0', '')
List of Tables in the Database
Customer
Old_Customer
Product


In [ ]:
# Example-2: Writing a CREATE TABLE Statement for a Product Table
# ProductCode, ProductName, Stock, Price, and Manufacturer
# ProductCode는 Primary Key
# CHECK: Stock는 0과 10000사이

cursor.execute("DROP TABLE IF EXISTS Product")

create_table_query = """
 CREATE TABLE Product(
  ProductCode CHAR(3) NOT NULL,
  ProductName VARCHAR(20),
  Stock INT,
  Price INT,
  Manufacturer VARCHAR(20),
  PRIMARY KEY (ProductCode),
  CONSTRAINT my_chk CHECK (Stock >= 0 AND Stock <= 10000)
 );
"""

cursor.execute(create_table_query)

cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

for table in tables:
  print(table)

cursor.execute("DESCRIBE Product")
rows = cursor.fetchall()

for row in rows:
  print(row)

('Customer',)
('Old_Customer',)
('Product',)
('ProductCode', 'char(3)', 'NO', 'PRI', None, '')
('ProductName', 'varchar(20)', 'YES', '', None, '')
('Stock', 'int', 'YES', '', None, '')
('Price', 'int', 'YES', '', None, '')
('Manufacturer', 'varchar(20)', 'YES', '', None, '')


In [ ]:
# ALTER TABLE STATEMENT
# 존재하는 테이블 구조를 수정
# 1. Add a column: ALTER TABLE Customer ADD Email VARCHAR(50);
# 2. Modify a column: ALTER TABLE Customer MODIFY Age INT NOT NULL;
# 3. Rename a column: ALTER TABLE Customer CHANGE OldName NewName VARCHAR(30);
# 4. Delete a column: ALTER TABLE Customer DROP COLUMN Job;

"""
ALTER TABLE table_name
ADD column_name data_type [NOT NULL] [DEFAULT default_value];
"""

"""
ALTER TABLE table_name
DROP COLUMN column_name;
"""

"""
ALTER TABLE table_name
ADD CONSTRAINT constraint_name constraint_definition
"""

# Example-3: Adding a New Column
# JoinDate column을 추가

alter_table_query = """
 ALTER TABLE Customer
 ADD COLUMN JoinDate VARCHAR(20);
"""

cursor.execute(alter_table_query)

cursor.execute("DESCRIBE Customer")
for row in cursor.fetchall():
  print(row)

('CustomerID', 'varchar(20)', 'NO', 'PRI', None, '')
('CustomerName', 'varchar(20)', 'NO', '', None, '')
('Age', 'int', 'YES', '', None, '')
('Grade', 'varchar(10)', 'NO', '', None, '')
('Job', 'varchar(20)', 'YES', '', None, '')
('Points', 'int', 'YES', '', '0', '')
('JoinDate', 'varchar(20)', 'YES', '', None, '')


In [ ]:
# Example-4: Dropping an Existing Column

alter_table_query = """
 ALTER TABLE Customer
 DROP COLUMN JoinDate;
"""

cursor.execute(alter_table_query)

cursor.execute("DESCRIBE Customer")
for row in cursor.fetchall():
  print(row)

('CustomerID', 'varchar(20)', 'NO', 'PRI', None, '')
('CustomerName', 'varchar(20)', 'NO', '', None, '')
('Age', 'int', 'YES', '', None, '')
('Grade', 'varchar(10)', 'NO', '', None, '')
('Job', 'varchar(20)', 'YES', '', None, '')
('Points', 'int', 'YES', '', '0', '')


In [ ]:
# Example-5: Adding a New Constraint
# Example-6: Dropping an Existing Constraint

alter_table_query = """
 ALTER TABLE Customer
 ADD CONSTRAINT CHK_AGE CHECK (AGE >= 20);
"""

# alter_table_query = """
#  ALTER TABLE Customer
#  DROP CONSTRAINT CHK_AGE;
# """

cursor.execute(alter_table_query)

cursor.execute("SHOW CREATE TABLE Customer")

for row in cursor.fetchall():
  print(row)

('Customer', "CREATE TABLE `Customer` (\n  `CustomerID` varchar(20) NOT NULL,\n  `CustomerName` varchar(20) NOT NULL,\n  `Age` int DEFAULT NULL,\n  `Grade` varchar(10) NOT NULL,\n  `Job` varchar(20) DEFAULT NULL,\n  `Points` int DEFAULT '0',\n  PRIMARY KEY (`CustomerID`),\n  CONSTRAINT `CHK_AGE` CHECK ((`AGE` >= 20))\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci")


In [ ]:
# Inserting Data: INSERT Statement
# INSERT: 행을 추가하는 것
# INTO: columns 선정
# VALUES: 값 설정
# 순서대로 들어감

"""
INSERT
INTO table_name (column_list)
VALUES (value_list)
"""

insert_query = """
 INSERT
 INTO Customer (CustomerID, CustomerName, Age, Grade, Job, Points)
 VALUES ('C001', 'Alice', 30, 'Gold', 'Engineer', 1000);
"""
cursor.execute(insert_query)


In [ ]:
# Example-7: Direct Row Insertion + Retrieval Check
# 1. C002, Bob, 28, Silver, Designer

insert_query = """
 INSERT
 INTO Customer (CustomerID, CustomerName, Age, Grade, Job)
 VALUES ('C002', 'Bob', 28, 'Silver', 'Designer')
"""
cursor.execute(insert_query)
cursor.execute("SELECT * FROM Customer;")

for row in cursor.fetchall():
  print(row)

('C001', 'Alice', 30, 'Gold', 'Engineer', 1000)
('C002', 'Bob', 28, 'Silver', 'Designer', 0)


In [ ]:
# Example-8: Direct Row Insertion + Retrieval Check
# single operation using executemany()
# executemany(Query, Data)
# 여러개 Data 리스트를 한번에 실행할 수 있음

first_insert_query = """
 INSERT
 INTO Customer (CustomerID, CustomerName, Age, Grade, Job, Points)
 VALUES (%s,%s, %s, %s, %s, %s);
"""

values = [('C003', 'Park', 21, 'VIP', 'Developer', 2000),
          ('C004', 'Kim', 25, 'VIP', 'Designer', 5000)]

cursor.executemany(first_insert_query, values)

cursor.execute("SELECT * FROM Customer;")

for row in cursor.fetchall():
  print(row)

('C001', 'Alice', 30, 'Gold', 'Engineer', 1000)
('C002', 'Bob', 28, 'Silver', 'Designer', 0)
('C003', 'Park', 21, 'VIP', 'Developer', 2000)
('C004', 'Kim', 25, 'VIP', 'Designer', 5000)


In [ ]:
# Insertion에서의 Subquery: SELECT를 이용해서 다른 테이블에서 값을 가져와서 Insert함

"""
INSERT
INTO table_name (column_list)
SELECT ...
FROM another_table
WHERE condition;
"""

cursor.execute("DROP TABLE IF EXISTS Old_Custmer")

create_query = """
 CREATE TABLE Old_Customer(
  CustomerID VARCHAR(20) NOT NULL,
  CustomerName VARCHAR(20) NOT NULL,
  Age INT,
  Grade VARCHAR(10) NOT NULL,
  Job VARCHAR(20),
  Points INT DEFAULT 0,
  PRIMARY KEY (CustomerID)
 );
"""

# cursor.execute(create_query)

insert_query = """
 INSERT
 INTO Old_Customer (CustomerID, CustomerName, Age, Grade, Job, Points)
 VALUES ('C005', 'Lee', 45, 'Gold', 'Engineer', 3000);
"""

cursor.execute(insert_query)

insert_query = """
 INSERT
 INTO Customer (CustomerID, CustomerName, Age, Grade, Job, Points)
 SELECT CustomerID, CustomerName, Age, Grade, Job, Points FROM Old_Customer WHERE Age > 40;
"""

cursor.execute(insert_query)

cursor.execute("SELECT * FROM Customer;")
for row in cursor.fetchall():
  print(row)

IntegrityError: 1062 (23000): Duplicate entry 'C005' for key 'Old_Customer.PRIMARY'

In [ ]:
# UPDATE STATEMENT
# 속성 값을 변경함
# WHERE 절이 없으면 모든 행을 업데이트함
# 의도적으로 모든 값을 변경하는거 아니면 WHERE 쓰는게 좋음

"""
 UPDATE table_name
 SET column1 = value1, column2 = value2, ...
 WHERE condition
"""

'\n UPDATE table_name\n SET column1 = value1, column2 = value2, ...\n WHERE condition\n'

In [ ]:
# Example-11: UPDATE Statement
# Grade를 변경 (CustomerID가 C001인 사람을 Gold -> Silver)

update_query = """
 UPDATE Customer
 SET Grade = 'Silver'
 WHERE CustomerID='C001'
"""

cursor.execute(update_query)

cursor.execute("SELECT * FROM Customer")
for row in cursor.fetchall():
  print(row)


('C001', 'Alice', 30, 'Silver', 'Engineer', 1000)
('C002', 'Bob', 28, 'Silver', 'Designer', 0)
('C003', 'Park', 21, 'VIP', 'Developer', 2000)
('C004', 'Kim', 25, 'VIP', 'Designer', 5000)


In [ ]:
# Example-12: UPDATE Statement
# 모든 Custoemr의 Points를 20% 인상하기

update_query = """
 UPDATE Customer
 SET Points = Points * 1.2;
"""

cursor.execute(update_query)
cursor.execute("SELECT * FROM Customer")
for row in cursor.fetchall():
  print(row)


('C001', 'Alice', 30, 'Silver', 'Engineer', 1200)
('C002', 'Bob', 28, 'Silver', 'Designer', 0)
('C003', 'Park', 21, 'VIP', 'Developer', 2400)
('C004', 'Kim', 25, 'VIP', 'Designer', 6000)


In [ ]:
# Example-13: UPDATE Statement에서의 subquery
# Job이 엔지니어인 Customer의 Points를 1000 인상 (subquery 사용하기)

# 1. SET 절에서 subquery 사용
"""
 UPDATE Table
 SET Column = (
  SELECT FROM WHERE ...
 )
 WHERE condition;
"""

# 2. WHERE 절에서 subquery 사용
"""
 UPDATE Table
 SET Column = Value
 WHERE Condition IN (SELECT FROM WHERE ...);
"""

# 3. UPDATE JOIN SET 사용
"""
 UPDATE Table AS T
 JOIN (
  SELECT FROM WHERE ...
 ) AS sub # 임시 테이블 생

"""

# STEP 1) Customer에서 Engineer의 Points를 가져와 temp_sub라는 임시 테이블 생성
# STEP 2) JOIN ON으로 ON 조건에 맞는 행들과 임시 테이블과 합쳐진 결과가 보여짐 (또 다른 임시 테이)
# STEP 3) 그리고 SET으로 임시임시 테이블에서 Points가져와 업데이트 수행

update_query = """
 UPDATE Customer
 JOIN (
  SELECT Job, Points + 1000 AS IncreasedPrice
  FROM Customer
  WHERE Job = 'Engineer'
 ) AS temp_sub
 ON Customer.Job = temp_sub.Job
 SET Customer.Points = temp_sub.IncreasedPrice
"""
cursor.execute(update_query)
cursor.execute("SELECT * FROM Customer")

for row in cursor.fetchall():
  print(row)


('C001', 'Alice', 30, 'Silver', 'Engineer', 2200)
('C002', 'Bob', 28, 'Silver', 'Designer', 0)
('C003', 'Park', 21, 'VIP', 'Developer', 2400)
('C004', 'Kim', 25, 'VIP', 'Designer', 6000)


In [ ]:
# DELETE STATEMENT
# WHERE 없으면 모든 행이 삭제될 것임

"""
DELETE
FROM table_name
WHERE condition;
"""



insert_query = """
 INSERT
 INTO Customer (CustomerID, CustomerName, Age, Grade, Job, Points)
 VALUES (%s, %s, %s, %s, %s, %s);
"""

values = [('C006', 'Kim', 22, 'Silver', 'Artist', 1500),
          ('C007', 'Park', 50, 'VIP', 'Artist', 7000),
           ('C008', 'Yoon', 60, 'VIP', 'Artist', 10000)]

cursor.executemany(insert_query, values)
cursor.execute("SELECT * FROM Customer")

for row in cursor.fetchall():
  print(row)

('C001', 'Alice', 30, 'Silver', 'Engineer', 2200)
('C002', 'Bob', 28, 'Silver', 'Designer', 0)
('C003', 'Park', 21, 'VIP', 'Developer', 2400)
('C004', 'Kim', 25, 'VIP', 'Designer', 6000)
('C006', 'Kim', 22, 'Silver', 'Artist', 1500)
('C007', 'Park', 50, 'VIP', 'Artist', 7000)
('C008', 'Yoon', 60, 'VIP', 'Artist', 10000)


In [ ]:
# Example-14: DELETE Statement
# C006 삭제하기

delete_query = """
 DELETE
 FROM Customer
 WHERE CustomerID = 'C006';
"""

cursor.execute(delete_query)
cursor.execute("SELECT * FROM Customer")

for row in cursor.fetchall():
  print(row)

('C001', 'Alice', 30, 'Silver', 'Engineer', 2200)
('C002', 'Bob', 28, 'Silver', 'Designer', 0)
('C003', 'Park', 21, 'VIP', 'Developer', 2400)
('C004', 'Kim', 25, 'VIP', 'Designer', 6000)
('C007', 'Park', 50, 'VIP', 'Artist', 7000)
('C008', 'Yoon', 60, 'VIP', 'Artist', 10000)


In [ ]:
# Example-15: DELETE Statement using a subquery
# Job이 Artist인 사람 삭제

# 1. DELETE FROM ... WHERE IN (SELECT FROM WHERE)
"""
 DELETE FROM table_name
 WHERE column IN (
  SELECT FROM WHERE
 )
"""

# 2. DELETE ... JOIN
"""
 DELETE T
 FROM Table AS T
 JOIN (
  SELECT FROM WHERE
 ) AS temp_sub
 ON C. = temp_sub.
"""

# 3. EXISTS
"""
 DELETE FROM table_name
 WHERE EXISTS (
  SELECT FROM WHERE
 )
"""

delete_query = """
 DELETE C
 FROM Customer AS C
 JOIN (
  SELECT Job AS J
  FROM Customer
  WHERE Job = 'Artist'
 ) AS temp_sub
 ON C.Job = temp_sub.J
"""

# delete_query = """
#  DELETE
#  FROM Customer
#  WHERE Job IN (
#   SELECT * FROM (
#     SELECT Job
#     FROM Customer
#     WHERE Job = 'Artist'
#   )
#  )
# """

cursor.execute(delete_query)

cursor.execute("SELECT * FROM Customer")

for row in cursor.fetchall():
  print(row)

('C001', 'Alice', 30, 'Silver', 'Engineer', 2200)
('C002', 'Bob', 28, 'Silver', 'Designer', 0)
('C003', 'Park', 21, 'VIP', 'Developer', 2400)
('C004', 'Kim', 25, 'VIP', 'Designer', 6000)


In [ ]:
# DELETE STATEMENT
# 테이블의 모든 행 삭제하기
# column names, data types, constraints만 남아 있음
# DROP과는 다름

"""
 DELETE FROM table_name;
"""

In [ ]:
# SELECT STATEMENT
# ALL: 중복된 행 포함해서 return (Default)
# DISTINCT: 중복된 행 있으면 제거해서 return
# AS: Column 별칭 지정
# Arithmetic Expression: *,+,-,/ with constants
# - 계산된 결과가 return
# Logical Operators: AND, OR, NOT
# Strings와 Dates는 무조건 single quotes로 되어야함

# LIKE: 패턴 매칭 해줌
# - %: 0개 이상 가져옴
# - _: 정확히 1개만 가져옴
# data%, %data, %data%, data___, __a%

# NULL Conditions
# IS NULL: NULL Value 인거 return
# IS NOT NULL: NULL value 아닌거 return
# NULL과 연산 수행하면 다 false 나올 것임

# Sorting
# ORDER BY
# ASC: 오름차순 정렬 (NULL은 맨 뒤에 나타남)
# DESC: 내림차순 정렬 (NULL은 맨 처음에 나타남)
# 여러 column 기준으로 정렬하면, 우선 순위를 명시해줌

# Aggregate
# COUNT(), SUM(), AVG(), MAX(), MIN()
# NULL은 무시함, WHERE 안에서 사용할 수 없음, SELECT 절이나, HAVING 절에서 사용 가능
# COUNT(*): NULL 갯수 포함해서 row 계산
# COUNT(column_name): NULL 갯수 제외하고 row 계산
# SUM과 AVG는 numeric에서만 사용가능

# Grouped Search
# GROUP BY: 집계함수랑 보통 같이 쓰이며 ..
# HAVING: 그룹화 이후 필터링

# Join Queries for Multiple Tables
# Table을 연결함
# Join 속성: 두 테이블을 연결하는 공통의 열
# - 속성명은 다름, 데이터 타입은 같아야함 (보통 외래키를 기준으로 조인함)
# FROM Table1 JOIN Order ON
# WHERE Table1.ID = Table2.CustomerID

# Example-16: SELECT Statement
# CustomerID, CustomerName, Grade, Customer를 가져올 것임
# all columns를 가져올 것임

select_query = """
 SELECT CustomerID, CustomerName, Grade
 FROM Customer
"""

cursor.execute(select_query)

for row in cursor.fetchall():
  print(row)

select_query = """
 SELECT *
 FROM Customer
"""

cursor.execute(select_query)

for row in cursor.fetchall():
  print(row)

('C001', 'Alice', 'Silver')
('C002', 'Bob', 'Silver')
('C003', 'Park', 'VIP')
('C004', 'Kim', 'VIP')
('C001', 'Alice', 30, 'Silver', 'Engineer', 2200)
('C002', 'Bob', 28, 'Silver', 'Designer', 0)
('C003', 'Park', 21, 'VIP', 'Developer', 2400)
('C004', 'Kim', 25, 'VIP', 'Designer', 6000)


In [ ]:
# Example-17: SELECT Statement
# customer name과 job 가져옴
# customer name과 ALL 키워드로 job 가져옴
# DISTINCT로 가져옴

# AS: 별칭 지정
# - Oracle에서는 ""로 함
# - MS-SQL에서는 ''로 함

select_query = """
 SELECT CustomerName, Job
 FROM Customer
"""

cursor.execute(select_query)

for row in cursor.fetchall():
  print(row)

select_query = """
 SELECT ALL CustomerName, Job
 FROM Customer
"""

cursor.execute(select_query)

for row in cursor.fetchall():
  print(row)

select_query = """
 SELECT DISTINCT CustomerName, Job
 FROM Customer
"""


cursor.execute(select_query)

for row in cursor.fetchall():
  print(row)

('Alice', 'Engineer')
('Bob', 'Designer')
('Park', 'Developer')
('Kim', 'Designer')
('Lee', 'Engineer')
('Alice', 'Engineer')
('Bob', 'Designer')
('Park', 'Developer')
('Kim', 'Designer')
('Lee', 'Engineer')
('Alice', 'Engineer')
('Bob', 'Designer')
('Park', 'Developer')
('Kim', 'Designer')
('Lee', 'Engineer')


In [ ]:
# Example-18: SELECT Statement
# AS로 Renaming하기

select_query = """
 SELECT CustomerName AS Name
 FROM Customer
"""


cursor.execute(select_query)

for row in cursor.fetchall():
  print(row)


# Renaming된 column 명을 확인하기 위한 코드
print([desc[0] for desc in cursor.description])

cursor.execute("SHOW columns in Customer")
columns = [col[0] for col in cursor.fetchall()]

print(columns)



('Alice',)
('Bob',)
('Park',)
('Kim',)
('Lee',)
['Name']
['CustomerID', 'CustomerName', 'Age', 'Grade', 'Job', 'Points']


In [ ]:
from tabulate import tabulate

# Example-19: CustomerName을 가져오고 Points에 1000을 더해서 가져옴

select_query = """
 SELECT CustomerName, Points + 1000 AS Score
 FROM Customer;
"""

cursor.execute(select_query)

rows = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]

print(tabulate(rows, headers=columns, tablefmt="psql"))

+----------------+---------+
| CustomerName   |   Score |
|----------------+---------|
| Alice          |    3200 |
| Bob            |    1000 |
| Park           |    3400 |
| Kim            |    7000 |
| Lee            |    5600 |
+----------------+---------+


In [ ]:
# Example-20: Conditional Query
# Points >= 4000, ID, Name, Age 가져오기
# VIP 또는 Points가 4000보다 큰거 가져오기
# Points 가 2000과 4000 사이인거 가져오기

select_query = """
 SELECT CustomerID, CustomerName, Age
 FROM Customer
 WHERE Points >= 4000
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

select_query = """
 SELECT *
 FROM Customer
 WHERE Grade = 'VIP' OR Points >= 4000;
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

select_query = """
 SELECT *
 FROM Customer
 WHERE Points >= 2000 AND Points <= 4000;
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

+--------------+----------------+-------+
| CustomerID   | CustomerName   |   Age |
|--------------+----------------+-------|
| C004         | Kim            |    25 |
| C005         | Lee            |    45 |
+--------------+----------------+-------+
+--------------+----------------+-------+---------+-----------+----------+
| CustomerID   | CustomerName   |   Age | Grade   | Job       |   Points |
|--------------+----------------+-------+---------+-----------+----------|
| C003         | Park           |    21 | VIP     | Developer |     2400 |
| C004         | Kim            |    25 | VIP     | Designer  |     6000 |
| C005         | Lee            |    45 | Gold    | Engineer  |     4600 |
+--------------+----------------+-------+---------+-----------+----------+
+--------------+----------------+-------+---------+-----------+----------+
| CustomerID   | CustomerName   |   Age | Grade   | Job       |   Points |
|--------------+----------------+-------+---------+-----------+----------

In [ ]:
# Example-21: Searching with LIKE

select_query = """
 SELECT CustomerName, Age, Grade, Points
 FROM Customer
 WHERE CustomerName LIKE 'A%' OR CustomerName LIKE 'B%'
"""


cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

+----------------+-------+---------+----------+
| CustomerName   |   Age | Grade   |   Points |
|----------------+-------+---------+----------|
| Alice          |    30 | Silver  |     2200 |
| Bob            |    28 | Silver  |        0 |
+----------------+-------+---------+----------+


In [ ]:
# Example-22: Searching with LIKE
# 정확히 3글자만 매칭시키기

select_query = """
 SELECT CustomerName, Age, Grade
 FROM Customer
 WHERE CustomerName LIKE '___'
"""


cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

+----------------+-------+---------+
| CustomerName   |   Age | Grade   |
|----------------+-------+---------|
| Bob            |    28 | Silver  |
| Kim            |    25 | VIP     |
| Lee            |    45 | Gold    |
+----------------+-------+---------+


In [ ]:
# Example-23: Searching with NULL Conditions

select_query = """
 SELECT CustomerName
 FROM Customer
 WHERE AGE IS NULL;
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))


select_query = """
 SELECT CustomerName
 FROM Customer
 WHERE AGE IS NOT NULL;
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

+----------------+
| CustomerName   |
|----------------|
+----------------+
+----------------+
| CustomerName   |
|----------------|
| Alice          |
| Bob            |
| Park           |
| Kim            |
| Lee            |
+----------------+


In [ ]:
# Example-24: Sorting Search Results
# 1. Age를 내림차순으로 정렬해서 Name, Grade, Age 가져오기
# 2. Points가 2000보다 크거나 같은거 가져오고, Age는 오름차순, 동일 값 있으면 Points를 내림차순으로 함

select_query = """
 SELECT CustomerName, Grade, Age
 FROM Customer
 ORDER BY Age ASC
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

select_query = """
 SELECT *
 FROM Customer
 WHERE Points >= 2000
 ORDER BY Age ASC, Points DESC
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

+----------------+---------+-------+
| CustomerName   | Grade   |   Age |
|----------------+---------+-------|
| Park           | VIP     |    21 |
| Kim            | VIP     |    25 |
| Bob            | Silver  |    28 |
| Alice          | Silver  |    30 |
| Lee            | Gold    |    45 |
+----------------+---------+-------+
+--------------+----------------+-------+---------+-----------+----------+
| CustomerID   | CustomerName   |   Age | Grade   | Job       |   Points |
|--------------+----------------+-------+---------+-----------+----------|
| C003         | Park           |    21 | VIP     | Developer |     2400 |
| C004         | Kim            |    25 | VIP     | Designer  |     6000 |
| C001         | Alice          |    30 | Silver  | Engineer  |     2200 |
| C005         | Lee            |    45 | Gold    | Engineer  |     4600 |
+--------------+----------------+-------+---------+-----------+----------+


In [ ]:
# Example-25: Sorting Search Results
# Points가 2000보다 크거나 같은 값 가져오기
# AGE로 오름차순, Points로 내림차순

# insert_query = """
#  INSERT
#  INTO Customer (CustomerID, CustomerName, Age, Grade, Job, Points)
#  VALUES ('C006', 'Yoon', 21, 'Silver', 'Engineer', 3000)
# """

# cursor.execute(insert_query)

select_query = """
 SELECT *
 FROM Customer
 WHERE Points >= 2000
 ORDER BY Age ASC, Points DESC
"""


cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

+--------------+----------------+-------+---------+-----------+----------+
| CustomerID   | CustomerName   |   Age | Grade   | Job       |   Points |
|--------------+----------------+-------+---------+-----------+----------|
| C006         | Yoon           |    21 | Silver  | Engineer  |     3000 |
| C003         | Park           |    21 | VIP     | Developer |     2400 |
| C004         | Kim            |    25 | VIP     | Designer  |     6000 |
| C001         | Alice          |    30 | Silver  | Engineer  |     2200 |
| C005         | Lee            |    45 | Gold    | Engineer  |     4600 |
+--------------+----------------+-------+---------+-----------+----------+


In [ ]:
# Example-26: Searching with Aggregate Functions
# 1. Points의 평균 계산하기
# 2. Total Points 계산하기 (Job이 Engineer인 사람)

select_query = """
 SELECT AVG(Points) AS AveragePoints
 FROM Customer
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

# AS에 공백있으면 '', ""로 묶어야함 (한 단어면 '', "", 없어도되고 있어도됨)
select_query = """
 SELECT SUM(Points) AS "Total Points"

 FROM Customer
 WHERE Job = 'Engineer'
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

+-----------------+
|   AveragePoints |
|-----------------|
|         3033.33 |
+-----------------+
+----------------+
|   Total Points |
|----------------|
|           9800 |
+----------------+


In [ ]:
# Example-27: Searching with Aggregation Functions
# insert_query = """
#  INSERT
#  INTO Customer (CustomerID, CustomerName, Age, Grade, Job, Points)
#  VALUES ('C007', 'Park', NULL, 'Gold', 'Artist', 1200)
# """

# cursor.execute(insert_query)

select_query = """
 SELECT COUNT(CustomerID) AS 'Total Customers'
 FROM Customer
"""


cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

select_query = """
 SELECT COUNT(Age) AS 'Customers with Age'
 FROM Customer
"""


cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

select_query = """
 SELECT COUNT(*) AS 'Total Rows'
 FROM Customer
"""


cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))



+-------------------+
|   Total Customers |
|-------------------|
|                 7 |
+-------------------+
+----------------------+
|   Customers with Age |
|----------------------|
|                    6 |
+----------------------+
+--------------+
|   Total Rows |
|--------------|
|            7 |
+--------------+


In [ ]:
# Example-28: Grouped Search
# 1. 직업 마다 Total Points를 가져오기

select_query = """
 SELECT Job, SUM(Points) AS TotalPoints
 FROM Customer
 GROUP BY Job
"""
cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))




+-----------+---------------+
| Job       |   TotalPoints |
|-----------+---------------|
| Engineer  |          9800 |
| Designer  |          6000 |
| Developer |          2400 |
| Artist    |          1200 |
+-----------+---------------+


In [ ]:
# Example-29: Grouped Search
# 1. Job 갯수 및 Points 최댓값 가져오기

select_query = """
 SELECT Grade, COUNT(DISTINCT Job) AS 'Number of Jobs', MAX(Points) AS 'Max Point'
 FROM Customer
 GROUP BY Grade
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))



+---------+------------------+-------------+
| Grade   |   Number of Jobs |   Max Point |
|---------+------------------+-------------|
| Gold    |                2 |        4600 |
| Silver  |                2 |        3000 |
| VIP     |                2 |        6000 |
+---------+------------------+-------------+


In [ ]:
# Example-30: Join Queries for Multiple Tables

cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS Grade_Info (
  Grade VARCHAR(10) PRIMARY KEY,
  DiscountRate INT
)
""")

conn.commit()

cursor.executemany(
    """
    INSERT INTO Grade_Info (Grade, DiscountRate) VALUES (%s, %s)
    """, [('Silver', 10), ('Gold', 20), ('VIP', 30)]
)

conn.commit()


IntegrityError: 1062 (23000): Duplicate entry 'Silver' for key 'Grade_Info.PRIMARY'

In [ ]:

# Grade_Info 테이블의 Grade를 참조하는 외래키 생성
# cursor.execute(
#     """
#     ALTER TABLE Customer
#     ADD CONSTRAINT fk_customer_grade
#     FOREIGN KEY (Grade)
#     REFERENCES Grade_Info(Grade);
#     """
# )

# 1. Customer와 Grade_Info table 연결하기

select_query = """
  SELECT *
  FROM Customer AS C
  JOIN Grade_Info AS J
  ON C.Grade = J.Grade
  ORDER BY DiscountRate ASC
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=columns, tablefmt="psql"))

+--------------+----------------+-------+---------+-----------+----------+---------+----------------+
| CustomerID   | CustomerName   |   Age | Grade   | Job       |   Points | Grade   |   DiscountRate |
|--------------+----------------+-------+---------+-----------+----------+---------+----------------|
| C001         | Alice          |    30 | Silver  | Engineer  |     2200 | Silver  |             10 |
| C002         | Bob            |    28 | Silver  | Designer  |        0 | Silver  |             10 |
| C006         | Yoon           |    21 | Silver  | Engineer  |     3000 | Silver  |             10 |
| C005         | Lee            |    45 | Gold    | Engineer  |     4600 | Gold    |             20 |
| C007         | Park           |       | Gold    | Artist    |     1200 | Gold    |             20 |
| C003         | Park           |    21 | VIP     | Developer |     2400 | VIP     |             30 |
| C004         | Kim            |    25 | VIP     | Designer  |     6000 | VIP    